<a href="https://colab.research.google.com/github/KhanutBJ/C.elegans_counter/blob/main/Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Object detection ด้วย FastAI**

**Credit: Dr. Titipat Achakulvisut**

Object detection การสร้างโมเดลมาเพื่อทำนายบริเวณที่มีวัตถุที่สนใจ และหลังจากนั้นทำนายว่าวัตถุในบริเวณที่สนใจเป็นวัตถุประเภทใด

ในตัวอย่างนี้เราจะนำชุดข้อมูลจาก Kaggle: [Monkey, Cat and Dog detection](https://www.kaggle.com/datasets/tarunbisht11/yolo-animal-detection-small) มาตรวจจับและแยกประเภทของข้าวสาลีกัน

ใน Notebook นี้เราจะทำการ
- โหลดชุดข้อมูล
- สร้างไฟล์ Object detection ในรูปแบบของ COCO format
- รัน Object detection ด้วย FastAI

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!kaggle datasets download -d tarunbisht11/yolo-animal-detection-small

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip yolo-animal-detection-small.zip -d yolo-animal-small

unzip:  cannot find or open yolo-animal-detection-small.zip, yolo-animal-detection-small.zip.zip or yolo-animal-detection-small.zip.ZIP.


In [ ]:
!git clone https://github.com/muellerzr/Practical-Deep-Learning-for-Coders-2.0

Cloning into 'Practical-Deep-Learning-for-Coders-2.0'...
remote: Enumerating objects: 1301, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 1301 (delta 23), reused 48 (delta 21), pack-reused 1227
Receiving objects: 100% (1301/1301), 86.34 MiB | 13.54 MiB/s, done.
Resolving deltas: 100% (810/810), done.


## **สร้าง COCO Dataset จากชุดข้อมูล CSV**

In [ ]:
import json
import ast
import pandas as pd
import os.path as op
from pathlib import Path
from PIL import Image

In [ ]:
annotation_df = pd.read_csv("yolo-animal-small/train.csv")
annotation_df.head()

FileNotFoundError: ignored

In [ ]:
Image.open("yolo-animal-small/yolo-animal-detection-small/train/cats_and_monkeys_097.jpg")

In [ ]:
annotation_df["bbox"] = annotation_df.apply(
    lambda r: [r["xmin"], r["ymin"], r["xmax"] - r["xmin"], r["ymax"] - r["ymin"]],
    axis=1
)
annotation_df["area"] = annotation_df.apply(lambda r: r["bbox"][2] * r["bbox"][3], axis=1)
annotation_df["id"] = annotation_df.filename.map(lambda x: Path(x).stem)

In [ ]:
annotation_df.head()

In [ ]:
img_df = annotation_df.groupby("id").first().reset_index()[["id", "width", "height"]]
img_df["file_name"] = img_df.id.map(lambda p: op.join("yolo-animal-small/yolo-animal-detection-small/train/", f"{p}.jpg"))

In [ ]:
img_df.head()

In [ ]:
images = img_df.to_dict("records")

categories = []
for i, c in enumerate(annotation_df["class"].unique()):
    categories.append({"supercategory": "animal", "id": i, "name": c})
cat2idx = {r["name"]: r["id"] for r in categories}

In [ ]:
categories

In [ ]:
annotation_df["category_id"] = annotation_df["class"].map(cat2idx)

In [ ]:
annotation_df.head()

In [ ]:
annotation_df.rename(columns={"id": "image_id"}, inplace=True)

In [ ]:
annotation_df["id"] = list(range(len(annotation_df)))
annotation_df["iscrowd"] = 0

In [ ]:
annotations = annotation_df[["id", "category_id", "iscrowd", "bbox", "image_id", "area"]].to_dict("records")

In [ ]:
coco_format = {
    "info": {"description": "Monkey Cat Dog Dataset from Kaggle", "url": "https://www.kaggle.com/datasets/tarunbisht11/yolo-animal-detection-small"},
    "licenses": [
        {"url": "https://www.kaggle.com/datasets/tarunbisht11/yolo-animal-detection-small"}
    ],
    "images": images,
    "annotations": annotations,
    "categories": categories
}

In [ ]:
json.dump(coco_format, open("yolo-animal-small/coco_format.json", "w"), indent=2)

In [ ]:
coco_format_load = json.load(open("yolo-animal-small/coco_format.json", "r"))

## **FastAI for Object detection**

Description: https://walkwithfastai.com/Object_Detection
Reference: https://youtu.be/5bSVug1YB3s?t=2067

In [ ]:
from fastbook import *
from fastai.vision.all import *

# add path ไปยัง Github ที่ clone มา
import sys
sys.path.append('Practical-Deep-Learning-for-Coders-2.0/Computer Vision/')
from imports import *

In [ ]:
# คำสั่งของ FastAI เพื่อได้ภาพ และ label boxes สำหรับแต่ละภาพ
imgs, label_bbox = get_annotations("yolo-animal-small/coco_format.json")
img2bbox = dict(zip(imgs, label_bbox))

In [ ]:
# place holder สำหรับ get_items
def get_train_images(noop = None):
    return imgs

In [ ]:
images = get_train_images()

In [ ]:
funcs = [lambda o: o, lambda o: img2bbox[o][0], lambda o: img2bbox[o][1]]
item_tfms = [Resize(224)]
batch_tfms = [Rotate(), Flip(), Dihedral(), Normalize.from_stats(*imagenet_stats)]

wheat_datablock = DataBlock(
    blocks=(ImageBlock, BBoxBlock, BBoxLblBlock),
    splitter=RandomSplitter(valid_pct=0.2),
    get_items=get_train_images,
    getters=funcs,
    item_tfms=item_tfms,
    batch_tfms=batch_tfms,
    n_inp=1
)

In [ ]:
dls = wheat_datablock.dataloaders("yolo-animal-small/yolo-animal-detection-small/train/")
dls.c = 3

In [ ]:
dls.show_batch(max_n=10)

In [ ]:
encoder = create_body(resnet34, pretrained=True)

In [ ]:
get_c(dls) # นับจำนวน Class

In [ ]:
# สร้าง RetinaNet ด้วย Resnet34 encoder 
retina_net = RetinaNet(encoder, get_c(dls), final_bias=-4)

In [ ]:
retina_net.box_regressor

In [ ]:
criterion = RetinaNetFocalLoss(scales=[1, 2**(-1/3), 2**(-2/3)], ratios=[1/2, 1, 2])

In [ ]:
def _retinanet_split(m):
    return L(
        m.encoder,
        nn.Sequential(m.c5top6, m.p6top7, m.merges, m.smoothers, m.classifier, m.box_regressor)
    ).map(params)

In [ ]:
learner = Learner(dls, retina_net, loss_func=criterion, splitter=_retinanet_split)

In [ ]:
learner.freeze()

In [ ]:
# มี error จาก FastAI
# ref: https://forums.fast.ai/t/typeerror-no-implementation-found-for-torch-nn-functional-smooth-l1-loss-on-types-that-implement-torch-function-class-fastai-torch-core-tensorimage-class-fastai-vision-core-tensorbbox/90897
# ที่สามารถแก้ได้ด้วยบรรทัดต่อไปนี้
TensorImage.register_func(torch.nn.functional.smooth_l1_loss, TensorImage, TensorBBox)
TensorMultiCategory.register_func(TensorMultiCategory.mul, TensorMultiCategory, TensorImage)
TensorImage.register_func(torch.nn.functional.binary_cross_entropy_with_logits, TensorImage, TensorMultiCategory)

In [ ]:
learner.fit_one_cycle(10, slice(1e-5, 1e-4))

## **Object detection ด้วย IceVision**

ใน Section นี้เราจะใช้ไลบรารี่ Icevision ในการสร้างโมเดล Object detection กัน

In [ ]:
import os.path as op
import pandas as pd
from icevision.all import *

In [ ]:
df = pd.read_csv("yolo-animal-small/train.csv")
df.rename(columns={"class": "label"}, inplace=True)

In [ ]:
df.to_csv("yolo-animal-small/train_annotation.csv", index=False)

In [ ]:
df.head()

In [ ]:
template_record = ObjectDetectionRecord()

class YoloSmallDataset(Parser):
    def __init__(self, template_record):
        super().__init__(template_record=template_record)
        self.df = pd.read_csv("yolo-animal-small/train_annotation.csv")
        self.class_map = ClassMap(list(self.df['label'].unique()))

    def __iter__(self) -> Any:
        for o in self.df.itertuples():
            yield o

    def __len__(self) -> int:
        return len(self.df)

    def record_id(self, o) -> Hashable:
        return o.filename

    def parse_fields(self, o, record, is_new):
        if is_new:
            record.set_filepath(op.join("yolo-animal-small/yolo-animal-detection-small/", "train", o.filename))
            record.set_img_size(ImgSize(width=o.width, height=o.height))
            record.detection.set_class_map(self.class_map)

        record.detection.add_bboxes([BBox.from_xyxy(o.xmin, o.ymin, o.xmax, o.ymax)])
        record.detection.add_labels([o.label])

In [ ]:
parser = YoloSmallDataset(template_record)
train_records, valid_records = parser.parse()

In [ ]:
len(train_records), len(valid_records)

In [ ]:
model_type = models.mmdet.retinanet
backbone = model_type.backbones.resnet50_fpn_1x

In [ ]:
train_tfms = tfms.A.Adapter([*tfms.A.aug_tfms(size=224, presize=512), tfms.A.Normalize()])
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(224), tfms.A.Normalize()])

In [ ]:
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

In [ ]:
train_dl = model_type.train_dl(train_ds, batch_size=8, num_workers=4, shuffle=True)
valid_dl = model_type.train_dl(train_ds, batch_size=8, num_workers=4, shuffle=False)

In [ ]:
samples = [train_ds[10] for _ in range(5)]
show_samples(samples, ncols=3)

In [ ]:
model = model_type.model(backbone=backbone(pretrained=True), num_classes=len(parser.class_map))

In [ ]:
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]
learner = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)

In [ ]:
learner.lr_find()

In [ ]:
learner.fine_tune(20, 1e-4, freeze_epochs=1)

In [ ]:
from icevision.models import *

In [ ]:
m = ClassMap(list(df['label'].unique()))
m.get_classes()

In [ ]:
# เซฟโมเดลด้วยคำสั่ง `save_icevision_checkpoint`
checkpoint_path = "monkeycatdog_det.pth"
save_icevision_checkpoint(
    model, 
    model_name='mmdet.retinanet', 
    backbone_name='resnet50_fpn_1x',
    classes=m.get_classes(),
    img_size=224,
    filename=checkpoint_path,
    meta={"icevision_version": "0.12.0"}
)

## **Example prediction**

ลองใช้โมเดลทำนายผลใน validation dataset `valid_ds`

In [ ]:
# โหลดโมเดลที่เซฟมาเพื่อทำนายผลใน validation set
model_loaded = model_from_checkpoint("monkeycatdog_det.pth")

In [ ]:
model_type = model_loaded["model_type"]
backbone = model_loaded["backbone"]
class_map = model_loaded["class_map"]
img_size = model_loaded["img_size"]
model_type, backbone, class_map, img_size

In [ ]:
model_type.show_results(model_loaded["model"], valid_ds, detection_threshold=0.4)